<a href="https://colab.research.google.com/github/fuyu-quant/ibl-qlora/blob/main/examples/ibl_llama2_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predictions by Llama2 for IBL

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
%%capture
!pip install bitsandbytes==0.40.0 transformers==4.31.0 peft==0.4.0 accelerate==0.21.0 sentencepiece==0.1.99 einops==0.6.1 evaluate==0.4.0 scikit-learn==1.2.2

In [7]:
from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('huggingface'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# トークナイザーとモデルの読み込み
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-hf")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-13b-hf",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    ),
    device_map={"":0}
)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
model = PeftModel.from_pretrained(
    model,
    "/content/drive/My Drive/ibl/checkpoint-1000/adapter_model/",
    device_map={"":0}
)
model.eval()

In [ ]:
from datasets import load_dataset

In [ ]:
import re

def ibl_test(input, true_data):
    prompt = f"### Instruction: {input}\n\n### Response: "

    # 推論の実行
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    output = re.search(r'### Response: (.+)', output).group(1)
    print('出力データ:\n', output)
    print('正解データ:\n', true_data)
    return

In [ ]:
dataset_name = 'ibl-ver1'
dataset = load_dataset(f"fuyu-quant/{dataset_name}", split='train')
input_data = dataset[0]['input']
output_data = dataset[0]['output']

In [ ]:
ibl_test(input_data, output_data)